In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# Project code

import libraries

In [3]:
import ee
import geemap

# Authenticate and initialize Google Earth Engine API
ee.Authenticate()
ee.Initialize(project='ee-geosynta')

In [4]:
# Loading the Sentinel-2 image collection.
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

# Defining a region of interest (ROI) as a rectangle.
ddn = ee.Geometry.Polygon(
        [ [77.70336408729965, 30.55530016444844],
          [77.70336408729965, 30.059720911875814],
          [78.39000959511215, 30.059720911875814],
          [78.39000959511215, 30.55530016444844]])

# Creating feature collections for different land cover classes (Forest, Water, Urban).
Forest = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([77.82375031706329, 30.41057315086976]),
            {
              "Class": 1,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([77.85670930143829, 30.4209356983649]),
            {
              "Class": 1,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([77.90690107605175, 30.37666338341595]),
            {
              "Class": 1,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([77.92338056823925, 30.358890289808382]),
            {
              "Class": 1,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([77.71007473341443, 30.39166406628676]),
            {
              "Class": 1,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Point([77.74784023634412, 30.377151842407613]),
            {
              "Class": 1,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Point([77.91354225400153, 30.271058411265834]),
            {
              "Class": 1,
              "system:index": "6"
            })])
Water = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([78.2798434702323, 30.061130035170603]),
            {
              "Class": 2,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([78.30302467855122, 30.102029593376386]),
            {
              "Class": 2,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([78.31244603810026, 30.12212076669471]),
            {
              "Class": 2,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([78.25071645278393, 30.14807448469483]),
            {
              "Class": 2,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([78.21213679672582, 30.19797006697161]),
            {
              "Class": 2,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Point([78.18002502114881, 29.959676905547642]),
            {
              "Class": 2,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Point([78.4358501452102, 30.436724213601458]),
            {
              "Class": 2,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Point([78.57105962376131, 30.37480549185768]),
            {
              "Class": 2,
              "system:index": "7"
            })])
Urban = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([77.88316335793434, 29.877719647778548]),
            {
              "Class": 3,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([77.89998617287574, 29.86283394173815]),
            {
              "Class": 3,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([77.8405932425956, 29.907000582231795]),
            {
              "Class": 3,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([77.81673231119912, 29.882742865797542]),
            {
              "Class": 3,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([78.0588360841092, 29.96415962545435]),
            {
              "Class": 3,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Point([78.07153902600373, 29.949286835454412]),
            {
              "Class": 3,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Point([77.54934511731233, 29.977245839821535]),
            {
              "Class": 3,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Point([77.53732882092561, 29.946014523071565]),
            {
              "Class": 3,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Point([78.07560275632922, 30.45737311413907]),
            {
              "Class": 3,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Point([78.08307002622668, 30.453599723809234]),
            {
              "Class": 3,
              "system:index": "9"
            })])

In [5]:
Map = geemap.Map(center=(25, 80), zoom=5)
Map.add_basemap("Esri.WorldImagery")

# filtering sentinel 2 image collection
s2_ddn_2023 = s2.filterDate('2023-01-01', '2023-12-31') \
                .filterBounds(ddn) \
                .median()
# select bands
bands = ['B2','B3','B4','B8']
s2_ddn_2023 = s2_ddn_2023.select(bands).addBands(s2_ddn_2023.normalizedDifference(['B8','B4']).rename('NDVI'))

displayparameters = {
    'min': 1000,
    'max': 4500,
    'bands': ['B8','B4','B3'],
}
Map.addLayer(s2_ddn_2023, displayparameters, "Image")

label = "Class"
training = Water.merge(Forest).merge(Urban)

# Extracting features
trainingImage = s2_ddn_2023.sampleRegions(
    collection = training,
    properties = [label],
    scale = 10
)

# Dividing samples into training and testing
trainingData = trainingImage.randomColumn()
trainSet = trainingData.filter(ee.Filter.lessThan('random', 0.8))
testSet = trainingData.filter(ee.Filter.greaterThanOrEquals('random', 0.8))

# Train a random forest classifier with specified parameters
classifier = ee.Classifier.smileRandomForest(
    numberOfTrees = 100,
    variablesPerSplit = 2,
    minLeafPopulation = 1,
    bagFraction = 0.5,
    seed = 0
)

classifier = ee.Classifier.smileRandomForest(100).train(trainSet, label, bands)

# Classify the image
classified = s2_ddn_2023.classify(classifier)

# display the result
Map.centerObject(ddn, 10)
Map.addLayer(classified, {'min': 1, 'max': 3, 'palette': ['green', 'blue', 'red']}, 'Land Cover')

print('Results of trained classifier', classifier.explain().getInfo())

# get info about the trained classifier
trainAccuracy = classifier.confusionMatrix()
print('Training error matrix', trainAccuracy.getInfo())
print('Training overall accuracy', trainAccuracy.accuracy().getInfo())

# test set accuracy
testSet = testSet.classify(classifier)
validationAccuracy = testSet.errorMatrix(label,'classification')
print('Validation error matrix', validationAccuracy.getInfo())
print('Validation accuracy', validationAccuracy.accuracy().getInfo())

Map

Results of trained classifier {'classes': [1, 2, 3], 'importance': {'B2': 22.384546485260767, 'B3': 17.307244897959187, 'B4': 17.339977324263042, 'B8': 43.013639455782325}, 'numberOfTrees': 100, 'outOfBagErrorEstimate': 0.18181818181818182, 'trees': ['n= 12\nnode), split, n, loss, yval, (yprob)\n* denotes terminal node\n1) root 12 8.0000 1 (0.0000 0.33333 0.33333 0.33333)\n 2) B8<=1773.63 4 0.0000 2 (0.0000 0.14286 0.71429 0.14286) *\n 3) B8>1773.63 8 4.0000 1 (0.0000 0.45455 0.090909 0.45455)\n  6) B3<=966.667 4 0.0000 1 (0.0000 0.71429 0.14286 0.14286) *\n  7) B3>966.667 4 0.0000 3 (0.0000 0.14286 0.14286 0.71429) *\n', 'n= 12\nnode), split, n, loss, yval, (yprob)\n* denotes terminal node\n1) root 12 8.0000 1 (0.0000 0.33333 0.33333 0.33333)\n 2) B3<=695.233 4 0.0000 1 (0.0000 0.71429 0.14286 0.14286) *\n 3) B3>695.233 8 4.0000 2 (0.0000 0.090909 0.45455 0.45455)\n  6) B8<=1919.15 4 0.0000 2 (0.0000 0.14286 0.71429 0.14286) *\n  7) B8>1919.15 4 0.0000 3 (0.0000 0.14286 0.14286 0.7142

Map(center=[30.307749864327892, 78.0466868412061], controls=(WidgetControl(options=['position', 'transparent_b…